# Butler current data access demo
(*from Jeff Carlings*)

In [ ]:
from lsst.daf.butler import Butler

In [ ]:
collection = 'LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195'
repo = 'dp2_prep'

butler = Butler(repo, collections=collection)

In [ ]:
butler.query_datasets('deep_coadd', limit=1)

In [ ]:
ra_cen = 53.076
dec_cen = -28.110

import lsst.sphgeom as sphgeom
region = sphgeom.Region.from_ivoa_pos("CIRCLE 53.076 -28.110 2.0")

In [ ]:
band = 'r'
query = "band.name = :band AND " \
        "visit_detector_region.region OVERLAPS POINT(:ra, :dec)"
print(query)

bind_params = {"band": band, "ra": ra_cen, "dec": dec_cen}
print(bind_params)

In [ ]:
dataset_refs = butler.query_datasets("source",
                                     bind=bind_params)
print(len(dataset_refs))
# dataset_refs

In [ ]:
import lsst.geom as geom
point = geom.SpherePoint(ra_cen*geom.degrees, dec_cen*geom.degrees)
skymap = butler.get("skyMap", skymap="lsst_cells_v2")
tract = skymap.findTract(point).tract_id
patch = skymap.findTract(point).findPatch(point).getSequentialIndex()
print(tract, patch)

In [ ]:
butler.registry.queryDatasetTypes('*consolidated_map*')

In [ ]:
maglim_refs = butler.query_datasets('deepCoadd_psf_maglim_consolidated_map_weighted_mean')

In [ ]:
maglim_refs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skyproj

In [ ]:
# maglim_g = butler.get(maglim_refs[0])
maglim_r = butler.get(maglim_refs[3])

In [ ]:
fig, ax = plt.subplots(figsize=(13, 8))
sp = skyproj.McBrydeSkyproj(ax=ax)
sp.draw_hspmap(maglim_r, lon_range=(280, 350))

# plt.xlabel("Right Ascension (deg)")
# plt.ylabel("Declination (deg)")
plt.colorbar(label="PSF Mag Limit (r-band)")
# plt.gca().invert_xaxis()

plt.show()

In [ ]:
maglim_g

In [ ]:
objtab = butler.get('object', dataId={'tract':2078, 'skymap':'lsst_cells_v2'},
                    parameters={'columns':['objectId', 'r_psfFlux', 'coord_ra', 'coord_dec']})

In [ ]:
objtab

In [ ]:
exptime_refs = butler.query_datasets('deepCoadd_exposure_time_consolidated_map_sum', band='r')

In [ ]:
exptime_map = butler.get(exptime_refs[0])

In [ ]:
# exptime_values = exptime_map.get_values_pos(x, y)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 8))
sp = skyproj.McBrydeSkyproj(ax=ax)
sp.draw_hspmap(exptime_map/30, lon_range=(280, 350))

# plt.xlabel("Right Ascension (deg)")
# plt.ylabel("Declination (deg)")
plt.colorbar(label="total exposure time (s; g-band)")
# plt.gca().invert_xaxis()

plt.show()